## ASSIGNMENT 2 - VIP

##### Filtering and Edge Detection

In [5]:
#packages
import numpy as np 
import cv2
#import matplotlib.pyplot as plt

sigma = [1,2,4,8]
#kernel_size 3x3? 5x5?

In [7]:

img = cv2.imread('mandrill.jpg', cv2.IMREAD_GRAYSCALE)  #includes greyscaling

cv2.imshow('image', img) #creates GUI
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
## Filter images and detect edges 
# implement, perform & evaluate

## Gaussian filtering 
# show result using σ = 1, 2, 4, 8 and explain in detail what can be seen.

## Gradient magnitude computation using Gaussian derivatives
# Use σ = 1, 2, 4, 8 pixels, and explain in detail what can be seen and how the results differ

## Laplacian-Gaussian filtering. You may implement this as a difference of Gaussians
# σ = 1, 2, 4, 8 pixels, and explain in detail what can be seen and how the results differ

## Canny (or similar) edge detection. 
# Describe the parameter values and their impact on the result. Select what you think is a set of good parameter values, apply, show and decribe the result